In [76]:
#coding=utf-8

In [77]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [78]:
outfile = open('touzijie-2.csv', 'a')

In [79]:
def unit(tag):
    expand_nob = tag.find_element_by_xpath(".//dl/dt[5]/a")
    expand_nob.click()
    time.sleep(2)
    driver.switch_to_window(driver.window_handles[-1])
    info = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "info")))
    financier = info.find_element_by_xpath(".//ul/li[1]/a").text.encode('utf-8')
    quantity = info.find_element_by_xpath(".//ul/li[3]").text.encode('utf-8')
    rounds = info.find_element_by_xpath(".//ul/li[4]/span[2]").text.encode('utf-8')
    industry_list = info.find_elements(By.XPATH, ".//ul/li[6]/a")
    industry = ""
    for n, i in enumerate(industry_list):
        if n == 0:
            industry += i.text.encode('utf-8')
        elif n > 0:
            industry += ';' + i.text.encode('utf-8')
    industry = "[" + industry + ']'
    date = info.find_element_by_xpath(".//ul/li[5]").text.encode('utf-8')
    title = info.find_element_by_xpath(".//h1").text.encode('utf-8')
    describe = driver.find_element_by_xpath("//*[@id='desc']/p").text.encode('utf-8')
    investors_list = info.find_elements(By.XPATH, ".//ul/li[2]/a")
    investors = ""
    for n, i in enumerate(investors_list):
        if n == 0:
            investors += i.text.encode('utf-8')
        elif n > 0:
            investors += ';' + i.text.encode('utf-8')
    investors = '[' + investors + ']'
    result = pd.DataFrame({'date': [date], 'title': [title], 'financier': [financier], 'quantity': [quantity], 'rounds': [rounds], 'industry': [industry], 'investors': [investors]})
    driver.close()
    driver.switch_to_window(driver.window_handles[-1])
    #print 'result: ', result
    return result

In [ ]:
url = 'http://zdb.pedaily.cn/inv/'
#username = ''
#password = ''
driver = webdriver.Firefox()
driver.set_window_size(1000, 1000)
driver.get(url)
wait = WebDriverWait(driver, 10)
time.sleep(1)
#element = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='loginName']")))
#element.send_keys(username)
#element = driver.find_element_by_xpath("//*[@id='pwd']")
#element.send_keys(password + Keys.ENTER)
time.sleep(2)
element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[4]/div/div[1]/div[3]/span/span")))
EVENT_LIST = driver.find_elements(By.XPATH, "//*[@id='inv-list']/li")[1:]
while True:
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[4]/div/div[1]/div[3]/span/span")))
        EVENT_LIST = driver.find_elements(By.XPATH, "//*[@id='inv-list']/li")[1:]
        DATA = pd.DataFrame({'date': [], 'title': [], 'financier': [], 'quantity': [], 'rounds': [], 'industry': [], 'investors': []})
        #current_url = driver.current_url
        for n, li in enumerate(EVENT_LIST):
            try:
                DATA = pd.concat([DATA, unit(li)])
                #print 'DATA: ', DATA
                time.sleep(2)
            except Exception as e:
                print e
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
        DATA.to_csv(outfile)
        print 'Results saved.'
        time.sleep(1)
        nextpage = driver.find_elements(By.XPATH, "/html/body/div[4]/div/div[1]/div[4]/div[3]/a")[-1]
        if nextpage.text == u'下一页':
            nextpage.click()
            time.sleep(5)
        else:
            break
    except Exception as e:
        print e  
        driver.quit()
driver.quit()
outfile.close()